In [45]:
from bs4 import BeautifulSoup
import re
import requests
import time

def apex():
    urls = [ f'https://apex-magazine.com/category/short-fiction/page/{i}' for i in range(1,11) ]
    soups = [BeautifulSoup(requests.get(url).text) for url in urls]
    stories = []
    for soup in soups:
        for article in soup.findAll('article'):
            date = article.find('span', attrs={'class':'elementor-post-date'}).text
            author = article.find('span', attrs={'class':'elementor-post-author'}).text
            title = article.find('h3').find('a').text
            href = article.find('h3').find('a').get('href')
            if '2022' in date:
                stories.append((title, author, href))
        # print(title, date)
    return stories

def bcs():
    url = 'https://www.beneath-ceaseless-skies.com/issues/2022/'
    soup = BeautifulSoup(requests.get(url).text)
    stories = []
    for issue in soup.findAll('div', attrs={'class':'post-body'}):
        for title in issue.findAll('div', attrs={'class':'issue-story-title'}):
            if title.previous_sibling.get('class') == ['issue-podcast-subheader']:
                continue
            author = title.next_sibling
            href = title.find('a').get('href')
            stories.append((title.find('a').text, author.text, href))

    return stories

def clarkesworld():
    url = 'https://clarkesworldmagazine.com/prior/?y=2022'
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    stories = set()
    for table in soup.findAll('div', attrs={'class':'index-table'}):
        # if table doesn't contain a p tag with the words fiction, keep going
        section = table.find('p', attrs={'class': 'section'})
        if section is None or section.text.strip().lower() != 'fiction':
            continue
        for storydiv in table.findAll('div'):
    #         print(storydiv.text)
            title = storydiv.find('p', attrs={'class':'story'})
            author = storydiv.find('p', attrs={'class':'byline'})
            if not title or not author:
                continue
            href = title.find('a').get('href')
            title = title.text.strip()
            author = author.text.replace('by', '').strip()
            if "Editor's Desk" in title:
                continue
            if "A Conversation with" in title:
                continue
            if "An Interview with" in title:
                continue
            stories.add((title, author, href))
    return list(stories)

def nature():
    url = 'https://www.nature.com/nature/articles?type=futures&year=2022'
    soup = BeautifulSoup(requests.get(url).text)
    stories = []
    for article in soup.findAll('article'):
        title = article.find('h3')
        author = article.find('span').text
        href = title.find('a').get('href')
        stories.append((title.text.strip(), author, 'https://nature.com'+href))
    return stories

def strange_horizons():
    url = 'http://strangehorizons.com/issue/2022/'
    r = requests.get(url)
    soup = BeautifulSoup(r.text)
    stories = set()
    for article in soup.findAll('div', attrs={'class':'article'}):
        title = article.find('div', attrs={'class':'title'})
        try:
            href = title.find('a').get('href')
        except:
            print(f'{title.text} has no href, skipping')
            continue
        author = article.find('div', attrs={'class':'author'}).text.replace('By:', '').strip()
        if re.match(r"http.+/fiction/.+/$", href):
            stories.add((title.text.strip(), author, href))
    return stories

def uncanny():
    urls = [
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-four/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-five/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-six/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-seven/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-eight/',
        'https://uncannymagazine.com/issues/uncanny-magazine-issue-forty-nine/',
    ]
    soups = [BeautifulSoup(requests.get(url).text) for url in urls]
    stories = []
    for soup in soups:
        for p in soup.find('div', attrs={'class':'more_fiction'}).findAll('p'):
            try:
                title, author = p.findAll('a')
                author = author.text
            except:
                title, author, translator = p.findAll('a')
                author = f'{author.text} (translated by {translator.text})'
            stories.append((title.text, author, title.get('href')))
    return stories

In [31]:
apex_stories = apex()


Message in a Vessel  
January 31, 2023 

Observations of a Small Object in Decaying Orbit  
December 6, 2022 

Brief Life Story of Lila  
October 25, 2022 

Nothing That Bleeds  
October 4, 2022 

Sky Boys  
August 16, 2022 

Creatures of the Dark Oasis  
July 5, 2022 

The Fruit of the Princess Tree  
May 17, 2022 

Nine Theories of Time  
March 22, 2022 

Lamia  
February 1, 2022 

It Happened in ‘Loontown  
January 4, 2022 


In [6]:
bcs_stories = bcs()

In [9]:
cw_stories = clarkesworld()

In [14]:
na_stories = nature()

In [38]:
sh_stories = strange_horizons()

Podcast: 6 June Poetry  has no href, skipping
Podcast: 9 May Poetry  has no href, skipping
The Author and the Critic III: Alison Rumfitt and Marisa Mercurio  has no href, skipping
Podcast: Broken Blue  has no href, skipping
Stories From The Radio: The Slide  has no href, skipping
Dad Explains Dimensions at Outback Steakhouse  has no href, skipping
The Gratitude of Mice  has no href, skipping
Wide-Shining Craters  has no href, skipping
Monday: Space Trash, Volume 1 by Jenn Woodall  has no href, skipping


In [44]:
un_stories = uncanny()